In [18]:
import spacy
import os 
from transformers import pipeline
nlp = spacy.load("en_core_web_sm")
summarizer = pipeline("summarization")


No model was supplied, defaulted to google-t5/t5-small and revision d769bba (https://huggingface.co/google-t5/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [19]:
def extract_clauses(text):
    doc = nlp(text)
    clauses = []
    for sent in doc.sents:
        if "section" in sent.text.lower() or "clause" in sent.text.lower():
            clauses.append(sent.text)
    return clauses

def summarize_clauses(clauses):
    summaries = []
    for clause in clauses:
        summary = summarizer(clause, max_length=130, min_length=30, do_sample=False)
        summaries.append(summary[0]['summary_text'])
    return summaries


In [20]:
import os

def main():
    current_dir = os.getcwd()
    file_path = os.path.join(current_dir, 'data', 'legal_document.txt')

    try:
        with open(file_path, 'r') as file:
            text = file.read().lower()  

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return
    
    clauses = extract_clauses(text)

    if not clauses:
        print("No clauses found in the document.")
        return

    summaries = summarize_clauses(clauses)
    for i, summary in enumerate(summaries):
        print(f"Clause {i+1} Summary:")
        print(summary)
        print("\n")

if __name__ == "__main__":
    main()



Your max_length is set to 130, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Clause 1 Summary:
service provider agrees to provide graphic design services as requested by client . fee refers to compensation for services as described in section 4 .


